In [ ]:
# import libaries 1

import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import SimpleITK as sitk
import radiomics 
import six,numpy as np
import os
import csv
import pandas as pd
import seaborn as sn
import pylab as pl
import scipy, importlib, pprint, matplotlib.pyplot as plt, warnings
import glmnet_python
import imblearn
import math
import scipy.stats

In [ ]:
# import libraries 2

from sklearn.preprocessing import StandardScaler, LabelEncoder, normalize
from sklearn.feature_selection import GenericUnivariateSelect, VarianceThreshold, mutual_info_classif, SelectFromModel, SelectKBest, SelectPercentile, f_classif, f_regression, mutual_info_regression, RFE, RFECV
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV, ridge_regression, SGDClassifier, RidgeClassifier, PassiveAggressiveClassifier, OrthogonalMatchingPursuit, MultiTaskLasso, LassoLars, Lars, RidgeCV, ElasticNet, ElasticNetCV, LogisticRegression, LassoLarsCV, LassoCV, LinearRegression, Ridge, Lasso, lasso_path, enet_path
from sklearn.model_selection import LeaveOneOut, cross_validate, cross_val_predict, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.svm import LinearSVC, SVC, SVR, NuSVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import mean_squared_error, accuracy_score, roc_curve, auc, roc_auc_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier, OutputCodeClassifier
from sklearn import linear_model
from sklearn.utils import resample
from sklearn.decomposition import PCA

from matplotlib import pyplot as plt
from radiomics import featureextractor,imageoperations
from pandas import Series, DataFrame 

from scipy import stats

from glmnet import glmnet
from glmnet import glmnet; from glmnetPlot import glmnetPlot
from glmnetPrint import glmnetPrint; from glmnetCoef import glmnetCoef; from glmnetPredict import glmnetPredict
from cvglmnet import cvglmnet; from cvglmnetCoef import cvglmnetCoef
from cvglmnetPlot import cvglmnetPlot; from cvglmnetPredict import cvglmnetPredict

from imblearn.under_sampling import RandomUnderSampler, TomekLinks, ClusterCentroids
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [ ]:
# Import csv files 

file = '/Users/hwang/Documents/research/1_MM/pyradiomics/scikit_radiomics_T1norm_noshape_MWU.csv'
data = pd.read_csv(file)
target = data['Class']
target = target.astype(np.float64)
feature = data.loc[:,data.columns != 'Class']
reader = csv.DictReader(open(file))

In [ ]:
# corrleation analysis to eliminate one of the highly correlated features 

feature = pd.DataFrame(feature)

imp_feat = feature

f = plt.subplots(figsize=(30,30))
corr_mat = imp_feat.corr().abs()
ax = sn.heatmap(corr_mat,annot=True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()
print(imp_feat.shape)

col_corr = set() # Set of all the names of deleted columns
threshold = 0.9
corr_matrix = imp_feat.corr()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
            colname = corr_matrix.columns[i] # getting the name of column
            col_corr.add(colname)
            if colname in imp_feat.columns:
                del imp_feat[colname] # deleting the column from the dataset
                
feature = imp_feat
print(feature.shape)

In [ ]:
# Train and test sets split 

name = reader.fieldnames[1:]
feature_train, feature_test, target_train, target_test = train_test_split(feature,target,test_size=0.25,random_state=6)

In [ ]:
# Data normalization

scaler = StandardScaler()
encoder = LabelEncoder()
feature_train = scaler.fit_transform(feature_train)
feature_test = scaler.fit_transform(feature_test)
encoder.fit(target_train)
encoder.fit(target_test)
target_train = encoder.fit_transform(target_train)
target_test = encoder.fit_transform(target_test)

In [ ]:
# For balancing unbalanced class (plot_2d_space)

def plot_2d_space(X, y, label='Classes'):  
    
    colors = ['#1F77B4', '#FF7F0E']
    markers = ['o', 's']
    for l, c, m in zip(np.unique(y), colors, markers):
        plt.scatter(
            X[y==l, 0],
            X[y==l, 1],
            c=c, label=l, marker=m
        )
    plt.title(label)
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
# For balancing unbalanced class (code)

# ros = RandomOverSampler()
ros = SMOTE(ratio='minority')
# ros = RandomUnderSampler(return_indices=True)
# ros = ClusterCentroids(ratio={0:10})
# ros = TomekLinks(return_indices=True, ratio='majority')

feature_train_ros, target_train_ros = ros.fit_sample(feature_train, target_train)
print(feature_train_ros.shape[0] - feature_train.shape[0], 'new random picked points')
# print('Removed indexes:', id_ros)

feature_train = feature_train_ros
target_train = target_train_ros

plot_2d_space(feature_train, target_train)

In [ ]:
# LASSO regularization: to determine alpha and important features 

lassocv = LassoCV(eps = 0.001, n_alphas = 100, alphas=None, cv=5, max_iter=10000000, normalize=True)
lassocv.fit(feature_train,target_train)

print("Alpha =", lassocv.alpha_)
train_model_score = lassocv.score(feature_train, target_train)
test_model_score = lassocv.score(feature_test, target_test)
coff_used = np.sum(lassocv.coef_ != 0)

print("mse =",mean_squared_error(target_test, lassocv.predict(feature_test)))
print("Number of features used:", coff_used)

feature_train_inframe = pd.DataFrame(feature_train)
features = pd.Series(lassocv.coef_, index=feature_train_inframe.columns)

for i,v in enumerate(np.nonzero([features])):
    print(v)
    
x = v.astype(int)
imp_feature_name = list()
imp_feature_val = list()
imp_feature_ind = list()
for i in range(x.shape[0]):
    ind = features.index[x[i]]
    feat = name[x[i]]
    val = features.values[x[i]]
    print(ind, feat, val)
    imp_feature_name.append(feat)
    imp_feature_val.append(val)
    imp_feature_ind.append(ind)

In [ ]:
# LASSO regularization: for mean square error plot with respect to alpha (lambda)

model = lassocv

m_log_alphas = -np.log10(model.alphas_)

plt.figure()
plt.plot(m_log_alphas, model.mse_path_, ':')
plt.plot(m_log_alphas, model.mse_path_.mean(axis=-1), 'k',
         label='Average across the folds', linewidth=2)
plt.axvline(-np.log10(model.alpha_), linestyle='--', color='k',
            label='alpha: CV estimate')

plt.legend()

plt.xlabel('-Log(Lambda)')
plt.ylabel('Mean Square Error')
plt.axis('tight')
plt.show()

In [ ]:
# LASSO regularization: correlation matrix among the selected features from lassocv

header = []
col = []
for i in range(x.shape[0]):
    # col.append(feature1.values[x[i]])
    col.append(feature.iloc[:,x[i]])
    header.append(name[x[i]])

col = np.array(col)
col = np.transpose(col)
frame = pd.DataFrame(col, columns = header)

ax = sn.heatmap(frame.corr(), annot=True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

In [ ]:
# LASSO regularization: list important features selected from LASSO regularization  

model = lassocv
sfm = SelectFromModel(model, threshold=0.00001)
sfm.fit(feature_train,target_train)

imp_feature_train = sfm.transform(feature_train)
imp_feature_test = sfm.transform(feature_test)

feature_train = imp_feature_train
feature_test = imp_feature_test

In [ ]:
# LASSO regularization: classification and cross validation using LASSO linear regression 

cv = KFold(5, shuffle=True, random_state=3)
# cv = LeaveOneOut()
# scores_loo = cross_val_score(model, feature_train, target_train, cv=cv) 
cvscore = cross_val_score(model, feature_train, target_train, scoring="r2", cv=cv)
# cvresult = cross_validate(lassocv, feature_train, target_train, cv=cv, return_train_score=True)
cvprediction = cross_val_predict(model, feature_train, target_train, cv=cv)
predicted = cvprediction.round()

for i in range(predicted.shape[0]):
    if predicted[i] > 1.0: 
        predicted[i] = 1
    if predicted[i] <= -1.0:
        predicted[i] = 0
            
accuracy = accuracy_score(target_train,abs(predicted))

tn_train, fp_train, fn_train, tp_train = confusion_matrix(target_train, abs(predicted)).ravel()
sensitivity_train = tp_train / (tp_train + fn_train)
specificity_train = tn_train / (tn_train + fp_train)
auctraining = roc_auc_score(target_train, abs(cvprediction))

# print("Validation score after 5-fold cross validation:", cvresult['test_score'].mean())
print("Classification accuracy on training set after 5-fold cross validation: {:2f}".format(accuracy))
print("Sensitivity on training set: {:2f}".format(sensitivity_train))
print("Specificity on training set: {:2f}".format(specificity_train))
print("Training score after 3-fold cross validation:", cvscore.mean())

target_test_pred = model.predict(feature_test)
asall_test = accuracy_score(target_test, target_test_pred.round())
auctest = roc_auc_score(target_test, target_test_pred)
predicted_test = target_test_pred.round()

for i in range(predicted_test.shape[0]):
    if predicted_test[i] > 1.0: 
        predicted_test[i] = 1
    if predicted_test[i] <= -1.0:
        predicted_test[i] = 0

tn_test, fp_test, fn_test, tp_test = confusion_matrix(target_test, abs(predicted_test)).ravel()
sensitivity_test = tp_test / (tp_test + fn_test)
specificity_test = tn_test / (tn_test + fp_test)


print("Classification accuracy on test set: {:2f}".format(asall_test))
print("Sensitivity on test set: {:2f}".format(sensitivity_test))
print("Specificity on test set: {:2f}".format(specificity_test))
print("Test score:", test_model_score)
print("Area under the ROC curve (training): {:2f}".format(auctraining))
print("Area under the ROC curve (test): {:2f}".format(auctest))

In [ ]:
# Prinpal Component Analysis (PCA)

pca = PCA(0.95)
pca.fit(feature_train)
pca.fit(feature_test)
imp_feature_train = pca.transform(feature_train)
imp_feature_test = pca.transform(feature_test)

n_pcs = pca.components_.shape[0]
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
most_important_names = [name[most_important[i]] for i in range(n_pcs)]
variance_ratios = [pca.explained_variance_ratio_[i] for i in range(n_pcs)]
# most_important_ind = [imp_feature_ind[most_important[i]] for i in range(n_pcs)]
dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}
df = pd.DataFrame(dic.items())
df['variance ratio'] = variance_ratios
print(df)

feature_train = imp_feature_train
feature_test = imp_feature_test

In [ ]:
# Prinpal Component Analysis (PCA): correlation matrix after PCA

header = []
col = []
for i in range(pca.components_.shape[0]):
    col.append(feature.iloc[:,most_important[i]])
    header.append(name[most_important[i]])

col = np.array(col)
col = np.transpose(col)
frame = pd.DataFrame(col, columns = header)

ax = sn.heatmap(frame.corr(), annot=True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

In [ ]:
# model construction using selected features: LR 

model = LogisticRegressionCV(cv=5, random_state=1, Cs=1)
model.fit(feature_train,target_train)

In [ ]:
# model construction using selected features: RF 

model = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=10)
model.fit(feature_train,target_train)

In [ ]:
# RF: determine parameters

n_estimators = [int(x) for x in np.linspace(start = 0, stop = 200, num = 11)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(0, 300, num = 11)]
max_depth.append(None)
min_samples_split = [1,2,4]
min_samples_leaf = [1,2,4]
bootstrap = [True]
oob_score = [True]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'oob_score': oob_score}

print(random_grid)

In [ ]:
# RF: use random grid to search for best hyperparameters

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=10, n_jobs = 1)

rf_random.fit(feature_train,target_train)
rf_random.best_params_

In [ ]:
def evaluate(model, feature_test,target_test):
    predictions = model.predict(feature_test)
    errors = abs(predictions - target_test)
    accuracy = (len(errors)-np.count_nonzero(errors))/(len(errors))
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
# RF: 

pred_train = model.predict(feature_train)
errors = abs(pred_train - target_train)
accuracy = (len(errors)-np.count_nonzero(errors))/(len(errors))
print('Model Performance')
print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
print('Accuracy = {:0.2f}%.'.format(accuracy))

In [ ]:
# RF: evaluate performance of random grid 

base_model = RandomForestClassifier(n_estimators = 100, random_state = 10)
base_model.fit(feature_train,target_train)
base_accuracy = evaluate(base_model, feature_test, target_test)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, feature_test, target_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [ ]:
# RF: Create a parameter grid based on the results from random search 

param_grid = {
    'bootstrap': [True],
    'max_depth': [90],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1,2],
    'min_samples_split': [2,4],
    'n_estimators': [180],
    'oob_score': [True]}

rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5, n_jobs = 1, verbose = 2)

In [ ]:
# RF: Fit the grid search to the data

grid_search.fit(feature_train,target_train)
grid_search.best_params_
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, feature_test, target_test)
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

In [ ]:
# RF: cross validation and result display using the model other than lasso 

model = best_grid

cv = 5
loo = LeaveOneOut()
loo.get_n_splits(feature_train)
scv = []
for train_idx, test_idx in loo.split(feature_train):
    model.fit(feature_train[train_idx,:], target_train[train_idx])
    s = model.score(feature_train[test_idx,:], target_train[test_idx])
    scv.append(s)
scores_loo = cross_val_score(model, feature_train, target_train, cv=cv) 

# kfold = KFold(3, shuffle=True, random_state=5)
# scores_kfold = cross_val_score(model, feature_train, target_train, cv=cv) 

cvprediction = cross_val_predict(model, feature_train, target_train, cv=cv, method = 'predict_proba')
predicted = abs(cvprediction[:,1])

for i in range(predicted.shape[0]):
    if predicted[i] > 1.0: 
        predicted[i] = 1
    if predicted[i] <= -1.0:
        predicted[i] = 0
        
accuracy = accuracy_score(target_train,predicted.round())

tn_train, fp_train, fn_train, tp_train = confusion_matrix(target_train, abs(predicted.round())).ravel()
sensitivity_train = tp_train / (tp_train + fn_train)
specificity_train = tn_train / (tn_train + fp_train)
auctraining = roc_auc_score(target_train, cvprediction[:,1])


# results based on cross-validation 
print("Classification accuracy on training set after 3-fold cross validation: {:2f}".format(accuracy))
print("Sensitivity on training set: {:2f}".format(sensitivity_train))
print("Specificity on training set: {:2f}".format(specificity_train))

# results without cross-validation 
target_train_pred = model.predict_proba(feature_train)
target_test_pred = model.predict_proba(feature_test)
asall_test = accuracy_score(target_test, target_test_pred[:,1].round())
auctest = roc_auc_score(target_test, target_test_pred[:,1])
predicted_test = target_test_pred[:,1].round()

for i in range(predicted_test.shape[0]):
    if predicted_test[i] > 1.0: 
        predicted_test[i] = 1
    if predicted_test[i] <= -1.0:
        predicted_test[i] = 0

tn_test, fp_test, fn_test, tp_test = confusion_matrix(target_test, abs(predicted_test)).ravel()
sensitivity_test = tp_test / (tp_test + fn_test)
specificity_test = tn_test / (tn_test + fp_test)


print("Classification accuracy on test set: {:2f}".format(asall_test))
print("Sensitivity on test set: {:2f}".format(sensitivity_test))
print("Specificity on test set: {:2f}".format(specificity_test))
print("Area under the ROC curve (training): {:2f}".format(auctraining))
print("Area under the ROC curve (test): {:2f}".format(auctest))

In [ ]:
# RF: display important features selected by Random Forest 

feature_imp = pd.Series(model.feature_importances_,index=feature.columns).sort_values(ascending=True)
# feature_imp = pd.Series(model.feature_importances_,index=frame.columns).sort_values(ascending=True)
sn.barplot(x = feature_imp, y = feature_imp.index)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

In [ ]:
# AUC: with 95% CI 

def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count, sample_weight):
    if sample_weight is None:
        return fastDeLong_no_weights(predictions_sorted_transposed, label_1_count)
    else:
        return fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight)


def fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank_weight(positive_examples[r, :], sample_weight[:m])
        ty[r, :] = compute_midrank_weight(negative_examples[r, :], sample_weight[m:])
        tz[r, :] = compute_midrank_weight(predictions_sorted_transposed[r, :], sample_weight)
    total_positive_weights = sample_weight[:m].sum()
    total_negative_weights = sample_weight[m:].sum()
    pair_weights = np.dot(sample_weight[:m, np.newaxis], sample_weight[np.newaxis, m:])
    total_pair_weights = pair_weights.sum()
    aucs = (sample_weight[:m]*(tz[:, :m] - tx)).sum(axis=1) / total_pair_weights
    v01 = (tz[:, :m] - tx[:, :]) / total_negative_weights
    v10 = 1. - (tz[:, m:] - ty[:, :]) / total_positive_weights
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def fastDeLong_no_weights(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating
              Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions, sample_weight=None):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:http://localhost:8888/notebooks/Documents/pyradiomics-master/notebooks/radiomics_lassocv.ipynb
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


alpha = .95
auc_train, auc_cov_train = delong_roc_variance(target_train,abs(cvprediction))
auc_test, auc_cov_test = delong_roc_variance(target_test,abs(predicted_test))

auc_std_train = np.sqrt(auc_cov_train)
auc_std_test = np.sqrt(auc_cov_test)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci_train = stats.norm.ppf(
    lower_upper_q,
    loc=auc_train,
    scale=auc_std_train)

ci_test = stats.norm.ppf(
    lower_upper_q,
    loc=auc_test,
    scale=auc_std_test)

ci_train[ci_train > 1] = 1
ci_test[ci_test > 1] = 1

print('AUC (train):', auc_train)
print('AUC COV (train):', auc_cov_train)
print('95% AUC CI (train):', ci_train)

print('AUC (test):', auc_test)
print('AUC COV (test):', auc_cov_test)
print('95% AUC CI (test):', ci_test)